In [ ]:
import pandas as pd
import pandas_profiling as pdpro
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from warnings import filterwarnings
from mpl_toolkits.mplot3d import Axes3D
import statsmodels.api as sm
import missingno as msno
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.neighbors import LocalOutlierFactor
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.preprocessing import scale
from sklearn.model_selection import ShuffleSplit, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import model_selection
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import tree
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve

In [ ]:
filterwarnings("ignore")

In [ ]:
df = pd.read_csv("../input/rental-price-of-indias-it-capital-pune-mh-ind/train.csv")

In [ ]:
pdpro.ProfileReport(df)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['furnishing'] = np.where(df['furnishing'] == 'Unfurnishe', 'Unfurnished', df['furnishing'])
df['furnishing'].unique()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.corr()

In [ ]:
df.bedroom.value_counts()

In [ ]:
df.bathrooms.value_counts()

In [ ]:
df.furnishing.value_counts()

In [ ]:
df.avalable_for.value_counts()

In [ ]:
df.floor_number.value_counts()

In [ ]:
df.parking.value_counts()

In [ ]:
df.propertyage.value_counts()

In [ ]:
df.powerbackup.value_counts()

In [ ]:
df.pooja_room.value_counts()

In [ ]:
df.servant_room.value_counts()

In [ ]:
df.store_room.value_counts()

In [ ]:
df.facing.value_counts()

In [ ]:
df.floor_type.value_counts()

In [ ]:
print(df.groupby(["bedroom","area"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","furnishing"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","floor_number"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","facing"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","floor_type"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","gate_community"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","parking"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","propertyage"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","study_room"])["rent"].mean())

In [ ]:
print(df.groupby(["bedroom","servant_room"])["rent"].mean())

In [ ]:
df.columns

In [ ]:
print(df.groupby(["bedroom","store_room"])["rent"].mean())

In [ ]:
print(df.isnull().sum())

In [ ]:
corrPearson = df.corr(method="pearson")
corrPearson

In [ ]:
corrSpearman = df.corr(method="spearman")
corrSpearman

In [ ]:
figure = plt.figure(figsize=(10,8))
sns.heatmap(corrPearson,annot=True, vmin=-1, vmax=+1)
plt.title("PEARSON")
plt.xlabel("COLUMNS")
plt.ylabel("COLUMNS")
plt.show()

In [ ]:
figure = plt.figure(figsize=(10,8))
sns.heatmap(corrSpearman,annot=True, vmin=-1, vmax=+1)
plt.title("Spearman")
plt.xlabel("COLUMNS")
plt.ylabel("COLUMNS")
plt.show()

In [ ]:
target = 'rent'

In [ ]:
num_features = [feature for feature in df.columns 
                            if df[feature].dtype != 'O' and feature != target]

In [ ]:
df[num_features].head()

In [ ]:
dis_features = [feature for feature in num_features if len(df[feature].unique()) < 20 and feature != target]

print(dis_features)

In [ ]:
con_features = [feature for feature in num_features if feature not in dis_features and feature != target]

print(con_features)

In [ ]:
for i in dis_features:
    print(i)
    print(shapiro(df[i]))

In [ ]:
for i in con_features:
    print(i)
    print(shapiro(df[i]))

In [ ]:
df.hist(figsize=(20,20),grid = True,sharex=False,bins=15)
plt.show()

In [ ]:
for feature in dis_features:
    df.groupby(feature)[target].mean().plot.box()
    
    plt.xlabel(feature)
    plt.ylabel(target)
    plt.title('{} vs log({})_median'.format(feature, target))
    plt.show()

In [ ]:
for feature in dis_features:
    df.groupby(feature)[target].mean().plot.bar()
    
    plt.xlabel(feature)
    plt.ylabel(target)
    plt.title('{} vs log({})_median'.format(feature, target))
    plt.show()

In [ ]:
rand = 0
for i in dis_features:
    rand = rand + 1
    if rand < 13: # total 14 columns but we don't need output columns
        figure = plt.figure(figsize=(10,8))
        sns.barplot(x= i ,y="rent",data=df)
        plt.show()
    else:
        break

In [ ]:
cat_features = [feature for feature in df.columns if feature not in num_features and feature != target]

df[cat_features].head()

In [ ]:
for feature in cat_features:
    print('{}: {} categories'.format(feature, len(df[feature].unique())))

In [ ]:
area = list(df['address'].astype(str))

area_new = []

for a in area:
    area_new.append(a[a.find(',') + 1: a.find(',', a.find(',') + 1)])
    
df['address'] = np.array(area_new)

In [ ]:
area_mean = {}

for area in np.unique(np.array(area_new)):
    area_mean[area] = df.loc[df['address'] == area]['rent'].mean()
    
df['address_num'] = df['address'].map(area_mean)

In [ ]:
df[target] = np.log(df[target])

In [ ]:
df.drop(['maintenance_amt', 'address'], axis = 1, inplace = True)

In [ ]:
df.drop(['bathrooms'], axis = 1, inplace = True)

In [ ]:
df

In [ ]:
dummy_df = pd.get_dummies(df, drop_first = True)

print(dummy_df.shape)
dummy_df.head()

In [ ]:
x = dummy_df.drop(['rent'], axis = 1)
y = dummy_df['rent']

In [ ]:
xTrain,xTest,yTrain,yTest = train_test_split(x,y,test_size=0.20,random_state=42)

In [ ]:
lm = LinearRegression().fit(xTrain,yTrain)

In [ ]:
ridge = Ridge().fit(xTrain,yTrain)

In [ ]:
lasso = Lasso().fit(xTrain,yTrain)

In [ ]:
knnr = KNeighborsRegressor().fit(xTrain,yTrain)

In [ ]:
rfr = RandomForestRegressor(random_state=42,verbose=False).fit(xTrain,yTrain)

In [ ]:
xgbr = XGBRegressor().fit(xTrain,yTrain)

In [ ]:
models = [lm,ridge,lasso,knnr,rfr,xgbr]

In [ ]:
for model in models:
    name = model.__class__.__name__
    R2CV = cross_val_score(model,xTest,yTest,cv=10,scoring="r2").mean()
    error = -cross_val_score(model,xTest,yTest,cv=10,scoring="neg_mean_squared_error").mean()
    print(name + ": ")
    print("-" * 30)
    print(R2CV)
    print(np.sqrt(error))
    print("-" * 30)

In [ ]:
r = pd.DataFrame(columns=["MODELS","R2CV"])

In [ ]:
for model in models:
    name = model.__class__.__name__
    R2CV = cross_val_score(model,xTest,yTest,cv=10,scoring="r2").mean()
    result = pd.DataFrame([[name,R2CV * 100]],columns=["MODELS","R2CV"])
    r = r.append(result)
figure = plt.figure(figsize=(30,10))
sns.barplot(x="R2CV",y="MODELS",data=r,color="g")
plt.xlabel("R2CV")
plt.ylabel("MODELS")
plt.xlim(-50,100)
plt.title("MODEL ACCURACY COMPARISON")
plt.show()

In [ ]:
ols = sm.OLS(yTrain,xTrain).fit()
print(ols.summary())